In [23]:
import json
import pysal as ps
import urllib
import numpy as np

In [24]:
from urlparse import urlparse
def parse_geojson(resource):
    gj = urlparse(resource)
    if not gj[1]:
        # either a serialized object or a local file
        try:
            return json.loads(gj[2])
        except:
            with open(gj[2]) as f:
                return json.load(f)    
    else:
        # remote url
        fs = urllib.urlopen(resource)
        return json.load(fs)
    return gj

r = parse_geojson("http://toae.org/pub/columbus.json")
ss = json.dumps(r)
s = parse_geojson(ss)
lf = parse_geojson("../examples/columbus.json")



In [25]:
lf == s and s == r

True

In [26]:
def contiguity_from_geojson(gj_input, wtype='rook'):
    WTYPE = wtype.upper()
    if WTYPE not in ['QUEEN', 'ROOK']:
        print "wtype must be 'QUEEN' or 'ROOK'"
    WTYPE =["QUEEN", 'ROOK'].index(WTYPE)+1
    collection = parse_geojson(gj_input)   
    n = len(collection)
    collection_type = collection['features'][0]['geometry']['type']
    if collection_type in ['Polygon', 'MultiPolygon']:
        polys = []
        ids = []
        i = 0
        for feature in collection['features']:
            polys.append(ps.cg.asShape(feature['geometry']))
            ids.append(i)
            i +=1
        pcollection = ps.cg.shapes.PolygonCollection(dict(zip(ids,polys)))
        neighbors = ps.weights.Contiguity.ContiguityWeightsPolygons(pcollection, wttype=WTYPE).w
        return ps.W(neighbors)
    else:
        print "GeoJSON feature type must be 'Polygon' or 'MultiPolygon'"
       
def queen_from_geojson(uri):
    return contiguity_from_geojson(uri, wtype='QUEEN')

def rook_from_geojson(uri):
    return contiguity_from_geojson(uri, wtype="ROOK")
    


In [27]:
wql = queen_from_geojson("../examples/columbus.json")
wqr = queen_from_geojson("http://toae.org/pub/columbus.json")
ss = json.dumps(json.load(urllib.urlopen("http://toae.org/pub/columbus.json")),separators=(',',':') )
wqs = queen_from_geojson(ss)

setting bounding box for FeatureCollection
setting bounding box for FeatureCollection
setting bounding box for FeatureCollection


In [16]:
wql.neighbors == wqr.neighbors == wqs.neighbors

True

In [21]:
# point FeatureCollection
baltimore = "../examples/baltimore.geojson"
collection = parse_geojson(baltimore)



In [31]:
def get_coordinates_from_collection(collection):
    coordinates = []
    for feature in collection['features']:
        coordinates.append(feature['geometry']['coordinates'])
    return np.array(coordinates)



In [32]:
get_coordinates_from_collection(collection)

array([[ 907. ,  534. ],
       [ 922. ,  574. ],
       [ 920. ,  581. ],
       [ 923. ,  578. ],
       [ 918. ,  574. ],
       [ 900. ,  577. ],
       [ 918. ,  576. ],
       [ 907. ,  576. ],
       [ 918. ,  562. ],
       [ 897. ,  576. ],
       [ 916. ,  569. ],
       [ 908. ,  573. ],
       [ 913. ,  566. ],
       [ 910. ,  574. ],
       [ 922. ,  569. ],
       [ 913. ,  536. ],
       [ 919. ,  533.5],
       [ 917.5,  535. ],
       [ 933. ,  548.5],
       [ 932.5,  552.5],
       [ 936.5,  548.5],
       [ 930. ,  542.5],
       [ 925. ,  545. ],
       [ 927. ,  552. ],
       [ 936. ,  554.5],
       [ 860. ,  554. ],
       [ 868. ,  550.5],
       [ 872.5,  543. ],
       [ 880.5,  544.5],
       [ 869. ,  551.5],
       [ 883. ,  538. ],
       [ 876. ,  541. ],
       [ 875.5,  549. ],
       [ 875. ,  550. ],
       [ 868. ,  545. ],
       [ 879. ,  552. ],
       [ 860. ,  555.5],
       [ 868. ,  556.5],
       [ 873. ,  549. ],
       [ 888.5,  545. ],


In [35]:
def get_properites_from_collection(collection, properties=[]):
    if not properties:
        properties = collection['features'][0]['properties'].keys()
    values = []
    for feature in collection['features']:
        values.append([feature['properties'][prop] for prop in properties])
    return properties, values    
    

In [36]:
b = get_properites_from_collection(collection)

In [37]:
len(b)

2

In [38]:
b[0]

[u'NSTOR',
 u'AC',
 u'CITCOU',
 u'NROOM',
 u'SQFT',
 u'PRICE',
 u'GAR',
 u'NBATH',
 u'Y',
 u'STATION',
 u'FIREPL',
 u'BMENT',
 u'X',
 u'DWELL',
 u'LOTSZ',
 u'AGE',
 u'PATIO']

In [39]:
b[1]

[[3.0,
  0.0,
  0.0,
  4.0,
  11.25,
  47.0,
  0.0,
  1.0,
  534.0,
  1,
  0.0,
  2.0,
  907.0,
  0.0,
  5.7,
  148.0,
  0.0],
 [2.0,
  1.0,
  1.0,
  7.0,
  28.92,
  113.0,
  2.0,
  2.5,
  574.0,
  2,
  1.0,
  2.0,
  922.0,
  1.0,
  279.51,
  9.0,
  1.0],
 [2.0,
  0.0,
  1.0,
  7.0,
  30.62,
  165.0,
  2.0,
  2.5,
  581.0,
  3,
  1.0,
  3.0,
  920.0,
  1.0,
  70.64,
  23.0,
  1.0],
 [2.0,
  1.0,
  1.0,
  7.0,
  26.12,
  104.3,
  2.0,
  2.5,
  578.0,
  4,
  1.0,
  2.0,
  923.0,
  1.0,
  174.63,
  5.0,
  1.0],
 [2.0,
  0.0,
  1.0,
  7.0,
  22.04,
  62.5,
  0.0,
  1.5,
  574.0,
  5,
  1.0,
  2.0,
  918.0,
  1.0,
  107.8,
  19.0,
  1.0],
 [3.0,
  0.0,
  1.0,
  6.0,
  39.42,
  70.0,
  1.0,
  2.5,
  577.0,
  6,
  1.0,
  3.0,
  900.0,
  1.0,
  139.64,
  20.0,
  1.0],
 [1.0,
  1.0,
  1.0,
  6.0,
  21.88,
  127.5,
  2.0,
  2.5,
  576.0,
  7,
  1.0,
  3.0,
  918.0,
  1.0,
  250.0,
  20.0,
  1.0],
 [3.0,
  0.0,
  1.0,
  8.0,
  36.72,
  53.0,
  0.0,
  1.5,
  576.0,
  8,
  0.0,
  0.0,
  907.0,
  1.

In [40]:
b = get_properites_from_collection(collection, ['AGE', 'AC'])

In [41]:
b

(['AGE', 'AC'],
 [[148.0, 0.0],
  [9.0, 1.0],
  [23.0, 0.0],
  [5.0, 1.0],
  [19.0, 0.0],
  [20.0, 0.0],
  [20.0, 1.0],
  [22.0, 0.0],
  [22.0, 1.0],
  [4.0, 1.0],
  [23.0, 0.0],
  [20.0, 1.0],
  [30.0, 0.0],
  [20.0, 1.0],
  [18.0, 0.0],
  [75.0, 0.0],
  [60.0, 0.0],
  [65.0, 0.0],
  [14.0, 0.0],
  [45.0, 0.0],
  [14.0, 0.0],
  [22.0, 0.0],
  [35.0, 0.0],
  [5.0, 1.0],
  [60.0, 1.0],
  [14.0, 1.0],
  [19.0, 1.0],
  [11.0, 0.0],
  [16.0, 1.0],
  [19.0, 1.0],
  [17.0, 0.0],
  [24.0, 0.0],
  [22.0, 0.0],
  [27.0, 0.0],
  [3.0, 1.0],
  [21.0, 1.0],
  [20.0, 0.0],
  [4.0, 1.0],
  [24.0, 0.0],
  [40.0, 0.0],
  [23.0, 0.0],
  [35.0, 0.0],
  [25.0, 0.0],
  [17.0, 1.0],
  [26.0, 1.0],
  [15.0, 1.0],
  [29.0, 0.0],
  [24.0, 1.0],
  [30.0, 0.0],
  [19.0, 0.0],
  [50.0, 0.0],
  [18.0, 1.0],
  [74.0, 0.0],
  [22.0, 1.0],
  [80.0, 0.0],
  [24.0, 1.0],
  [20.0, 1.0],
  [7.0, 1.0],
  [47.0, 0.0],
  [50.0, 0.0],
  [45.0, 0.0],
  [48.0, 1.0],
  [50.0, 0.0],
  [50.0, 0.0],
  [48.0, 0.0],
  [55.0, 0.0],
